## Task

In this compulsory task you will clean the country column and parse the date column in the **store_income_data_task.csv** file.

In [6]:
# Load up store_income_data.csv
import pandas as pd
import numpy as np 
import fuzzywuzzy
from fuzzywuzzy import process
from datetime import datetime

df = pd.read_csv('store_income_data_task.csv')
df.head()

,id,store_name,store_email,department,income,date_measured,country
0,1,"Cullen/Frost Bankers, Inc.",NaN,Clothing,$54438554.24,4-2-2006,United States/
1,2,Nordson Corporation,NaN,Tools,$41744177.01,4-1-2006,Britain
2,3,"Stag Industrial, Inc.",NaN,Beauty,$36152340.34,12-9-2003,United States
3,4,FIRST REPUBLIC BANK,ecanadine3@fc2.com,Automotive,$8928350.04,8-5-2006,Britain/
4,5,Mercantile Bank Corporation,NaN,Baby,$33552742.32,21-1-1973,United Kingdom


1. Take a look at all the unique values in the "country" column. Then, convert the column to lowercase and remove any trailing white spaces.

In [7]:
countries = df['country'].unique()
print (f'There are {len(countries)} unique countries')
countries


There are 77 unique countries


array(['United States/', 'Britain', ' United States', 'Britain/',
       ' United Kingdom', 'U.K.', 'SA ', 'U.K/', 'America',
       'United Kingdom', nan, 'united states', ' S.A.', 'England ', 'UK',
       'S.A./', 'ENGLAND', 'BRITAIN', 'U.K', 'U.K ', 'America/', 'SA.',
       'S.A. ', 'u.k', 'uk', ' ', 'UK.', 'England/', 'england',
       ' Britain', 'united states of america', 'UK/', 'SA/', 'SA',
       'England.', 'UNITED KINGDOM', 'America.', 'S.A..', 's.a.', ' U.K',
       ' United States of America', 'Britain ', 'England', ' SA',
       'United States of America.', 'United States of America/',
       'United States.', 's. africasouth africa', ' England',
       'United Kingdom ', 'United States of America ', ' UK',
       'united kingdom', 'AMERICA', 'America ',
       'UNITED STATES OF AMERICA', ' S. AfricaSouth Africa', 'america',
       'S. AFRICASOUTH AFRICA', 'Britain.', '/', 'United Kingdom.',
       'United States', ' America', 'UNITED STATES', 'sa',
       'United States

In [19]:
df['country'] = df['country'].str.lower()
df['country'] = df['country'].str.strip()
df['country'] = df['country'].str.strip('""/.''')


print(f"There are {len(countries)} unique countries")
countries

There are 4 unique countries


array(['United Kingdom', 'South Africa', 'United States', 'unknown'],
      dtype=object)

2. Note that there should only be three separate countries. Eliminate all variations, so that 'South Africa', 'United Kingdom' and 'United States' are the only three countries.

In [14]:
matches = fuzzywuzzy.process.extract("u.k", countries, limit=10, 
                                     scorer=fuzzywuzzy.fuzz.token_sort_ratio)

matches


[('u.k', 100),
 ('uk', 40),
 ('united kingdom', 35),
 ('s.a', 33),
 ('united states', 25),
 ('unknown', 20),
 ('s. africasouth africa', 17),
 ('united states of america', 15),
 ('britain', 0),
 ('sa', 0)]

In [ ]:
def replace_matches (df, column, string_to_match, min_ratio = 90):
    # All unique strings from column being cleaned i.e.country.
    strings = df['country'].unique()
    matches = fuzzywuzzy.process.extract(string_to_match, strings, limit=10, 
                                    scorer=fuzzywuzzy.fuzz.token_sort_ratio)
    close_matches = [matches[0] for matches in matches  
                     if matches[1] >= min_ratio] 
    rows_with_matches = df[column].isin(close_matches)
    df.loc[rows_with_matches, column] = string_to_match


In [16]:
df.replace(['uk', 'britain', 'u.k', 'england', 'united kingdom'], 
           'United Kingdom', inplace=True)
df.replace(['s.a', 'sa', 's. africasouth africa'], 'South Africa', 
           inplace=True)
df.replace(['america', 'united states of america', 'united states'], 
           'United States', inplace=True)
df.replace('', np.nan, inplace=True) 
df.dropna(inplace=True)


countries = df['country'].unique()
print(f"There are {len(countries)} unique countries")
countries

There are 4 unique countries


array(['United Kingdom', 'South Africa', 'United States', 'unknown'],
      dtype=object)

3. Create a new column called `days_ago` in the DataFrame that is a copy of the 'date_measured' column but instead it is a number that shows how many days ago it was measured from the current date. Note that the current date can be obtained using `datetime.date.today()`.

In [36]:
df['date_measured'] = pd.to_datetime(df['date_measured'], format='%d-%m-%Y')

# Check the datatype of the 'date_measured' column after conversion
print(df['date_measured'].dtype)

# Calculate the number of days ago each measurement was taken
current_date = pd.to_datetime('today').normalize()
df['days_ago'] = (current_date - df['date_measured']).dt.days
df.head()

datetime64[ns]


,id,store_name,store_email,department,income,date_measured,country,days_ago
3,4,FIRST REPUBLIC BANK,ecanadine3@fc2.com,Automotive,$8928350.04,2006-05-08,united kingdom,6573
5,6,"Auburn National Bancorporation, Inc.",ccaldeyroux5@dion.ne.jp,Grocery,$69798987.04,1999-09-19,united kingdom,8996
6,7,"Interlink Electronics, Inc.",orodenborch6@skyrock.com,Garden,$22521052.79,2001-06-08,south africa,8368
9,10,"Synopsys, Inc.",lcancellieri9@tmall.com,Electronics,$44091294.62,2006-07-11,united kingdom,6509
15,16,New Home Company Inc. (The),nhinchcliffef@whitehouse.gov,Shoes,$90808764.99,1993-04-21,united kingdom,11338
